In [75]:
#Install skweak using pip command
!pip install skweak
#Install SpaCy library
!pip install spacy
#Install ftfy library
!pip install ftfy
#install pip snips-nlu-parsers library 
!pip install snips-nlu-parsers
#install pip hmm model 
!pip install --upgrade --user hmmlearn



         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.


         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.       


         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.


         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.       

In [76]:
#Download en_core_web_sm and en_core_web_md trained pipelines of SpaCy.
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_md 



         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.

2022-08-14 21:06:32.958582: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-08-14 21:06:32.958625: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart

2022-08-14 21:06:40.972072: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-14 21:06:40.972113: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-14 21:06:40.972135: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cube-cd70192a-dd08-4511-9626-437097f6c716-86b65795f-6mgw4): /proc/driver/nvidia/version does not exist
     |████████████████████████████████| 42.8 MB 1.5 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [78]:
import spacy
import sys
import skweak
from tqdm import tqdm
from datetime import datetime
from dateutil.parser import parse as dateparser
import spacy, re
from skweak import heuristics, gazetteers, aggregation, utils
import tarfile
import numpy as np 
import pandas as pd


In [79]:
import os,glob
folder_path = '/project/Court_listings_dataset/Court_listings_dataset/'
nlp = spacy.load('en_core_web_md')
for filename in glob.glob(os.path.join(folder_path, '*.txt')):
    with open(filename, 'r') as f:
        text = f.read()
        text = text.split('\n')
        #text = text.split('.')
        for sentence in text:
            doc = nlp(sentence)
        #print (filename)
            skweak.utils.display_entities(doc) 
    break
    

### Step 1: Labelling functions

In [80]:
#Import CUSTOM COURT LISTINGS WEAK ANNOTATORS from Mishcon

import sys

sys.path.append('/project/Mishcon_skweak/Court_listings_NER/weak_labelling/')

from court_listings_labelling import *
from court_listings_annotations import *
from court_listing_labelling_functions_token_level import *
from court_listings_bert_lf import *
from court_listings_convert_vlexLabels_to_court_listingsLabels import *
from court_listings_gen_ann import *
from court_listings_gen_json import *
from court_listings_LFs_tokens import *
from court_listings_matcher import * 
from court_listings_posting_dynamodb import *
from court_listings_unittest_output import *


In [81]:
############################################
# MODELS TO USE FOR
# CUSTOM COURT LISTINGS WEAK ANNOTATORS
############################################

nlp_spacy_md = spacy.load("en_core_web_md")
nlp_spacy_sm = spacy.load("en_core_web_sm")

In [82]:
from court_listings_matcher import spacy_matcher
spacy_matcher = spacy_matcher()

### Court Name  

### court_names_matcher -BEST

In [83]:
# We create the labelling function
court_name_detector = skweak.heuristics.FunctionAnnotator("court_name_detector", court_names_matcher)

# We run the function on the full corpus

for filename in glob.glob(os.path.join(folder_path, '*.txt')):
    with open(filename, 'r') as f:
        text = f.read()
        text = text.split('\n')
        #text = text.split('.')
        for sentence in text:
            doc = court_name_detector(nlp(sentence)) 
                # Show an example
            skweak.utils.display_entities(doc,"court_name_detector")
    break

In [84]:
# We create the labelling function
court_name_detector = skweak.heuristics.FunctionAnnotator("court_name_detector", court_names_matcher)

# df
d = {'court': []}

# We run the function on the full corpus
i = 0
for filename in glob.glob(os.path.join(folder_path, '*.txt')):
    with open(filename, 'r') as f:
        text = f.read()
        text = text.split('\n')
        for sentence in text:
            doc = court_name_detector(nlp(sentence)) 

# create spans to extrat entity by using skweak functions

            spans = skweak.utils.get_spans(doc,["court_name_detector"])

            #print(spans)
            #skweak.utils.display_entities(doc,"court_name_detector")
    
            d['court'] = d['court']+spans

#create dataframe 
df_court = pd.DataFrame(data=d)
df_court




,court
0,"(Business, and, Property, Courts)"
1,"(Commercial, Court)"
2,"(Interim, Applications, List, (, ChD, ))"
3,"(Upper, Tribunal, (, Tax, and, Chancery, Chamb..."
4,"(Business, and, Property, Courts)"
...,...
15488,"(Commercial, court)"
15489,"(Business, and, Property, Courts)"
15490,"(Commercial, Court)"
15491,"(Interim, Applications, List, (, ChD, ))"


### claimant, defendant, case name, case number

### claimant_and_defendant_generator_regex

In [85]:
# We create the labelling function
claimant_defendant_detector = skweak.heuristics.FunctionAnnotator("claimant_defendant_detector", claimant_and_defendant_generator_regex)

# We run the function on the full corpus
#for filename in glob.glob(os.path.join(folder_path, '*.txt')):
    #with open(filename, 'r') as f:
        #text = f.read()
        #text = text.split('\n')
for sentence in text:
    doc = claimant_defendant_detector(nlp(sentence)) 
            # Show an example
    skweak.utils.display_entities(doc,"claimant_defendant_detector")
break 

SyntaxError: 'break' outside loop (<ipython-input-85-d8b7e9ffc593>, line 13)

In [86]:
# We create the labelling function
claimant_defendant_detector = skweak.heuristics.FunctionAnnotator("claimant_defendant_detector", claimant_and_defendant_generator_regex)

d = {'Listing': [],'Claimant': [],'Defendant': [],'case_name': []}
# We run the function on the full corpus
i = 0

for filename in glob.glob(os.path.join(folder_path, '*.txt')):
    with open(filename, 'r') as f:
        text = f.read()
        text = text.split('\n')
        for sentence in text:
            doc = claimant_defendant_detector(nlp(sentence)) 

# create spans to extrat 4 entities by using skweak functions
   
            spans = skweak.utils.get_spans(doc,["claimant_defendant_detector"])
            #print(spans)
            #skweak.utils.display_entities(doc,"claimant_detector")
            if len(spans) > 2:
                d['Listing'] = d['Listing']+[spans[0]]
                d['Claimant'] = d['Claimant']+[spans[1]]
                d['Defendant'] = d['Defendant']+[spans[2]]
                d['case_name'] = d['case_name']+[[spans[1],spans[2]]]

#create dataframe 
df_claimant_defendant_listing_case_name = pd.DataFrame(data=d)
print(df_claimant_defendant_listing_case_name)



                   Listing                                           Claimant  \
0     (BL-2020, -, 001465)                                                (A)   
1     (BL-2020, -, 001491)                                                (A)   
2     (HC-2017, -, 002440)                                              (Day)   
3     (HC-2017, -, 002378)  (Powis, Street, Estates, (, No, ., 3, ), Limited)   
4     (BL-2020, -, 000899)                            (Evergen, Systems, Ltd)   
...                    ...                                                ...   
3676  (HT-2019, -, 000209)                    (Marble, Acquisitions, Limited)   
3677  (HT-2019, -, 000351)                           (EA, Chiverton, Limited)   
3678  (BL-2019, -, 002365)                 (CBPE, Capital, Fund, VIII, A, LP)   
3679  (BR-2019, -, 001173)                                                (A)   
3680  (PT-2019, -, 000423)                                          (Rowland)   

                           

### judges

### judge_generator_matcher -BEST

In [89]:
# We create the labelling function
judge_detector = skweak.heuristics.FunctionAnnotator("judge_detector", judge_generator_matcher)

# We run the function on the full corpus
#for filename in glob.glob(os.path.join(folder_path, '*.txt')):
   # with open(filename, 'r') as f:
       # text = f.read()
        #text = text.split('\n')
for sentence in text:
    doc = judge_detector(nlp(sentence)) 
            # Show an example
    skweak.utils.display_entities(doc,"judge_detector")
break 


SyntaxError: 'break' outside loop (<ipython-input-89-2f33208c7035>, line 13)

In [90]:
# We create the labelling function
judge_detector = skweak.heuristics.FunctionAnnotator("judge_detector", judge_generator_matcher)

d = {'judge': []}
# We run the function on the full corpus
i = 0
for filename in glob.glob(os.path.join(folder_path, '*.txt')):
    with open(filename, 'r') as f:
        text = f.read()
        text = text.split('\n')
        for sentence in text:
            doc = judge_detector(nlp(sentence)) 

# create spans to extrat entity by using skweak functions
   
            spans = skweak.utils.get_spans(doc,["judge_detector"])
            #print(spans)
            #skweak.utils.display_entities(doc,"judge_detector")
            d['judge'] = d['judge']+spans

#create dataframe 
df_judge = pd.DataFrame(data=d)
df_judge

,judge
0,"(MR, JUSTICE, MEADE, –, Remotely, via, Skype)"
1,"(MASTER, KAYE)"
2,"(DEPUTY, MASTER, SMITH)"
3,"(of, MASTER, SHUMAN)"
4,"(MASTER, CLARK)"
...,...
5341,"(INSOLVENCY, AND, COMPANIES, COURT, JUDGE, MUL..."
5342,"(INSOLVENCY, AND, COMPANIES, COURT, JUDGE, BUR..."
5343,"(MR, JUSTICE, FANCOURT)"
5344,"(DEPUTY, MASTER, HANSEN, –, Remotely, via, Skype)"


### Date

### date_court_format_generator -BEST

In [91]:
# We create the labelling function
date_detector_1 = skweak.heuristics.FunctionAnnotator("date_detector_1", date_court_format_generator)

# We run the function on the full corpus
#for filename in glob.glob(os.path.join(folder_path, '*.txt')):
    #with open(filename, 'r') as f:
        #text = f.read()
        #text = text.split('\n')
for sentence in text:
    doc = date_detector_1(nlp(sentence)) 
            # Show an example
    skweak.utils.display_entities(doc,"date_detector_1")
break 

SyntaxError: 'break' outside loop (<ipython-input-91-d8d3862aa03f>, line 13)

In [92]:
# We create the labelling function
date_detector_1 = skweak.heuristics.FunctionAnnotator("date_detector_1", date_court_format_generator)


d = {'date': []}

# We run the function on the full corpus
i = 0
for filename in glob.glob(os.path.join(folder_path, '*.txt')):
    with open(filename, 'r') as f:
        text = f.read()
        text = text.split('\n')
        for sentence in text:
            doc = date_detector_1(nlp(sentence)) 

# create spans to extrat entity by using skweak functions
   
            spans = skweak.utils.get_spans(doc,["date_detector_1"])
            #print(spans)
            #skweak.utils.display_entities(doc,"date_detector_1")
            d['date'] = d['date']+spans

#create dataframe 
df_date = pd.DataFrame(data=d)
df_date

,date
0,"(Wednesday, 23, September, 2020)"
1,"(Wednesday, 23, September, 2020)"
2,"(Wednesday, 23, September, 2020)"
3,"(Wednesday, 23, September, 2020)"
4,"(Wednesday, 23, September, 2020)"
...,...
5166,"(Tuesday, 14, April, 2020)"
5167,"(Tuesday, 14, April, 2020)"
5168,"(Tuesday14, April, 2020)"
5169,"(Tuesday, 14, April, 2020)"


### time_court_format_generator - BEST

In [93]:
# We create the labelling function
time_detector_1 = skweak.heuristics.FunctionAnnotator("time_detector_1", time_court_format_generator)

# We run the function on the full corpus
#for filename in glob.glob(os.path.join(folder_path, '*.txt')):
    #with open(filename, 'r') as f:
        #text = f.read()
        #text = text.split('\n')
for sentence in text:
    doc = time_detector_1(nlp(sentence)) 
            # Show an example
    skweak.utils.display_entities(doc,"time_detector_1")
break 

SyntaxError: 'break' outside loop (<ipython-input-93-3a18c1132b0e>, line 13)

In [94]:
# We create the labelling function
time_detector_1 = skweak.heuristics.FunctionAnnotator("time_detector_1", time_court_format_generator)


d = {'time': []}
# We run the function on the full corpus
i = 0
for filename in glob.glob(os.path.join(folder_path, '*.txt')):
    with open(filename, 'r') as f:
        text = f.read()
        text = text.split('\n')
        for sentence in text:
            doc = time_detector_1(nlp(sentence)) 

# create spans to extrat entity by using skweak functions
   
            spans = skweak.utils.get_spans(doc,["time_detector_1"])
            #print(spans)
            #skweak.utils.display_entities(doc,"time_detector_1")
            d['time'] = d['time']+spans

#create dataframe 
df_time = pd.DataFrame(data=d)
df_time

,time
0,"(10:30, AM)"
1,"(11:00, AM)"
2,"(10:30, AM)"
3,"(10:30, AM)"
4,"(11:00, AM)"
...,...
3633,"(12:00, PM)"
3634,"(11:30, AM)"
3635,"(10:30, AM)"
3636,"(11:00, AM)"


### Type of hearing

### hearing_type_generator

In [95]:
# We create the labelling function
hearing_type_generator_fun = skweak.heuristics.FunctionAnnotator("hearing_type_generator_fun", hearing_type_generator)

# We run the function on the full corpus
#for filename in glob.glob(os.path.join(folder_path, '*.txt')):
    #with open(filename, 'r') as f:
        #text = f.read()
        #text = text.split('\n')
for sentence in text:
    doc = hearing_type_generator_fun(nlp(sentence))
            # Show an example
    skweak.utils.display_entities(doc, "hearing_type_generator_fun")
break 

SyntaxError: 'break' outside loop (<ipython-input-95-f08a0e3d8ae1>, line 13)

In [96]:
# We create the labelling function
hearing_type_generator_fun = skweak.heuristics.FunctionAnnotator("hearing_type_generator_fun", hearing_type_generator)


d = {'hearing type': []}
# We run the function on the full corpus
i = 0
for filename in glob.glob(os.path.join(folder_path, '*.txt')):
    with open(filename, 'r') as f:
        text = f.read()
        text = text.split('\n')
        for sentence in text:
            doc = hearing_type_generator_fun(nlp(sentence)) 

# create spans to extrat entity by using skweak functions
   
            spans = skweak.utils.get_spans(doc,["hearing_type_generator_fun"])
            #print(spans)
            #skweak.utils.display_entities(doc,"hearing_type_generator_fun")
            d['hearing type'] = d['hearing type']+spans

#create dataframe 
df_hearing = pd.DataFrame(data=d)
df_hearing

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname HMRC identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname IN identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname SAR identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  war

,hearing type
0,(Home)
1,(Courts)
2,"(Procedure, rules)"
3,(Offenders)
4,(Search)
...,...
36526,"(Website, queries)"
36527,"(Citizen, and, business, advice)"
36528,"(GOV, UK)"
36529,(GOV.UK)


### Step 3: Dataframe 

In [24]:
#create new dataframe to include all the entities 
dfs = [df_court, df_claimant_defendant_listing_case_name, df_judge, df_date, df_time, df_hearing]

#create one single dataframe 
df_merged = pd.concat(dfs, join='outer', axis=1)
df_merged

,court,Listing,Claimant,Defendant,case_name,judge,date,time,hearing type
0,"(Business, and, Property, Courts)","(BL-2020, -, 001465)",(A),(B),"[(A), (B)]","(MR, JUSTICE, MEADE, –, Remotely, via, Skype)","(Wednesday, 23, September, 2020)","(10:30, AM)",(Home)
1,"(Commercial, Court)","(BL-2020, -, 001491)",(A),(B),"[(A), (B)]","(MASTER, KAYE)","(Wednesday, 23, September, 2020)","(11:00, AM)",(Courts)
2,"(Interim, Applications, List, (, ChD, ))","(HC-2017, -, 002440)",(Day),(Salmon),"[(Day), (Salmon)]","(DEPUTY, MASTER, SMITH)","(Wednesday, 23, September, 2020)","(10:30, AM)","(Procedure, rules)"
3,"(Upper, Tribunal, (, Tax, and, Chancery, Chamb...","(HC-2017, -, 002378)","(Powis, Street, Estates, (, No, ., 3, ), Limited)","(Cradick, Retail, LLP)","[(Powis, Street, Estates, (, No, ., 3, ), Limi...","(of, MASTER, SHUMAN)","(Wednesday, 23, September, 2020)","(10:30, AM)",(Offenders)
4,"(Business, and, Property, Courts)","(BL-2020, -, 000899)","(Evergen, Systems, Ltd)","(Sespco, Limited)","[(Evergen, Systems, Ltd), (Sespco, Limited)]","(MASTER, CLARK)","(Wednesday, 23, September, 2020)","(11:00, AM)",(Search)
...,...,...,...,...,...,...,...,...,...
36526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(Website, queries)"
36527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(Citizen, and, business, advice)"
36528,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(GOV, UK)"
36529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(GOV.UK)


In [25]:
#create new dataframe for data cleaning 
df_reduce = df_merged.copy()
df_reduce["hearing type"] = df_reduce["hearing type"].astype(str).str.replace("(", "")


In [26]:
#create bins to remove all the unrelated texts

bins = ['Home', 'Courts', 'Procedure rules', 'Offenders',
        'Search', 'Menu ≡', 'Top', 'Admiralty Court', 'Business list', 'Chancery Appeals',
        'Commercial Court', 'Competition list', 'Financial List', 'Insolvency and companies list', 'Intellectual Property List',
        'Interim Applications List (ChD)', 'Property Trusts and Probate List', 'Revenue List', 'Technology and Construction Court',  'Upper Tribunal (Tax and Chancery Chamber) List',  '¬© Crown copyright', 'A' , 
        'A v', 'Vladimir Anatolevich Chemukhin for', 'Susan.Woolley@justice.gov.uk', 'Adjourned Bankruptcy Petitions', 'Accessibility', 'Adjudication Enforcement', '-', 'Afterwards', 'Admiralty Application', 'Admiralty CMC', 'Admiralty PTR', 'Admiralty Trial', 
        'Allied Wallet Limited', 'All affected parties shall have permission to apply in respect of any matter concerning the petition.' , 
        'All hearings are adjourned generally with liberty to restore on an urgent basis. If your matter is urgent please e-mail' , 
        'AMTRUST EUROPE LIMITED' , 'and' , 'and MR JUSTICE BUTCHER' ,  'AND UPON' ,  'AND UPON THE COURT being satisfied that the hearing of the petitions cannot presently be conducted remotely and that satisfactory” , “arrangements to ensure safety cannot be put in place such that it is necessary that the Petitions should be adjourned as set',  
        'Appeal' , 'Appeals' , 'Application' ,  'Application & Directions Hearing' ,  'Application and Consequentials/Further Arguments Hearings” , “Application by Andrew Jozef Frejek' ,  'Application by Ashridge Investments Limited' , 'Application by Dell Emerging Markets (EMEA) Ltd and others for', 
        'Application by Folgender Holdings Limited and another for', 'Application by Intergral Petroleum S.A.for', 'Application by Lakatamia Shipping Company Limited for', 
        'Application by Langham Park Homes', 'Application by Navigator Equities Limited and', 'Application by Nicholas Alan Vincent for', 'Application by Sahara Energy Resource Limited for', 'Application by Super-Max Offshore Holdings Limited', 
        'Application by Yuzu Hair Beauty for', 'Application for adjournment', 'Application Hearing & Chancery Trial', 
        'Application Hearing & Costs CMC', 'Application Hearing & Third Party Debt Order', 'as', 'ase contact', 'at', 'At 10:30 AM -', 
        'At 10:30 AM¬†¬†- Remotely via Skype~*', 'Available types:', 'Top ‚Üë', 'Website queries','GOV UK','Help','¬© Crown copyright','Contacts','Privacy','Copyright','GOV.UK','© Crown copyright','Cookies','Young witnesses']

#create new cleaned dataframe 
df_reduce = df_reduce[df_reduce["hearing type"].str.contains('|'.join(bins)) == False].reset_index(drop = True)
df_reduce

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,court,Listing,Claimant,Defendant,case_name,judge,date,time,hearing type
0,"(Admiralty, Court, (, QBD, ))","(BL-2019, -, 000207)","(Roman, Production, Limited)","(Silver, Levene, LLP)","[(Roman, Production, Limited), (Silver, Levene...","(DEPUTY, MASTER, LLOYD)","(Wednesday, 23, September, 2020)","(02:30, PM)",Daily court lists
1,"(Business, list, (, ChD, ))","(BL-2018, -, 000586)",(Battenberg),(Phillips),"[(Battenberg), (Phillips)]","(MR, JUSTICE, MILES)","(Wednesday, 23, September, 2020)","(02:00, PM)",Daily court lists
2,"(Commercial, Court, (, QBD, ))","(BR-2015, -, 000780)",(A),"(B, (, skype, ))","[(A), (B, (, skype, ))]","(of, MASTER, CLARK)","(Monday, 8, June, 2020)","(10:30, AM)",*Remote hearing via Skype for Business*
3,"(High, Court, of, Justice)","(BL-2020, -, 000647)",(A),(B),"[(A), (B)]","(MR, JUSTICE, MARCUS, SMITH, –, Via, Skype, *)","(Monday, 8, June, 2020)","(03:00, PM)",Directions Hearing
4,"(Commercial, court)","(PT-2020, -, 000130)",(Ghuman),(Ghuman),"[(Ghuman), (Ghuman)]","(MR, JUSTICE, ANDREW, BAKER)","(Monday, 8, June, 2020)","(10:30, AM)",*Remote hearing via Skype for Business*
...,...,...,...,...,...,...,...,...,...
12300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Legal aid
12301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mercantile Court
12302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pilot forms
12303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Traffic Enforcement Centre


In [27]:
#remove null values 

inds = df_reduce[['court', 'Listing', 'Claimant', 'Defendant', 'case_name', 'judge','date','time']].isnull().all(axis=1)
df_reduce = df_reduce.loc[~inds, :]
df_reduce

,court,Listing,Claimant,Defendant,case_name,judge,date,time,hearing type
0,"(Admiralty, Court, (, QBD, ))","(BL-2019, -, 000207)","(Roman, Production, Limited)","(Silver, Levene, LLP)","[(Roman, Production, Limited), (Silver, Levene...","(DEPUTY, MASTER, LLOYD)","(Wednesday, 23, September, 2020)","(02:30, PM)",Daily court lists
1,"(Business, list, (, ChD, ))","(BL-2018, -, 000586)",(Battenberg),(Phillips),"[(Battenberg), (Phillips)]","(MR, JUSTICE, MILES)","(Wednesday, 23, September, 2020)","(02:00, PM)",Daily court lists
2,"(Commercial, Court, (, QBD, ))","(BR-2015, -, 000780)",(A),"(B, (, skype, ))","[(A), (B, (, skype, ))]","(of, MASTER, CLARK)","(Monday, 8, June, 2020)","(10:30, AM)",*Remote hearing via Skype for Business*
3,"(High, Court, of, Justice)","(BL-2020, -, 000647)",(A),(B),"[(A), (B)]","(MR, JUSTICE, MARCUS, SMITH, –, Via, Skype, *)","(Monday, 8, June, 2020)","(03:00, PM)",Directions Hearing
4,"(Commercial, court)","(PT-2020, -, 000130)",(Ghuman),(Ghuman),"[(Ghuman), (Ghuman)]","(MR, JUSTICE, ANDREW, BAKER)","(Monday, 8, June, 2020)","(10:30, AM)",*Remote hearing via Skype for Business*
...,...,...,...,...,...,...,...,...,...
5196,"(Technology, and, Construction, Court, (, QBD, ))",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Remote Hearing via Skype for Business
5197,"(Commercial, court)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,TCC Trial
5198,"(Business, and, Property, Courts)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,tcc.listing@justice.gov.uk
5199,"(Commercial, Court)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Remote Hearing via Skype for Business


In [28]:
#remove "()" from dataframe 

df_reduce["time"] = df_reduce["time"].astype(str).str.replace("(", "")
df_reduce["date"] = df_reduce["date"].astype(str).str.replace("(", "")
df_reduce["judge"] = df_reduce["judge"].astype(str).str.replace("(", "")
df_reduce["case_name"] = df_reduce["case_name"].astype(str).str.replace("(", "")
df_reduce["Defendant"] = df_reduce["Defendant"].astype(str).str.replace("(", "")
df_reduce["Claimant"] = df_reduce["Claimant"].astype(str).str.replace("(", "")
df_reduce["Listing"] = df_reduce["Listing"].astype(str).str.replace("(", "")
df_reduce["court"] = df_reduce["court"].astype(str).str.replace("(", "")


<ipython-input-28-6667c491e074>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduce["time"] = df_reduce["time"].astype(str).str.replace("(", "")
<ipython-input-28-6667c491e074>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduce["date"] = df_reduce["date"].astype(str).str.replace("(", "")
<ipython-input-28-6667c491e074>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [29]:
#save the uncleaned dataframe into csv format 

df_reduce.to_csv('df_reduce_final.csv')


### Create Cleaned Dataset

In [30]:
#create new dataframe for data cleaning 

df_reduce_all = df_merged.copy()
df_reduce_all["hearing type"] = df_reduce_all["hearing type"].astype(str).str.replace("(", "")


In [31]:
#create bins to remove all the unrelated texts

bins = ['Home', 'Courts', 'Procedure rules', 'Offenders',
        'Search', 'Menu ≡', 'Top', 'Admiralty Court', 'Business list', 'Chancery Appeals',
        'Commercial Court', 'Competition list', 'Financial List', 'Insolvency and companies list', 'Intellectual Property List',
        'Interim Applications List (ChD)', 'Property Trusts and Probate List', 'Revenue List', 'Technology and Construction Court',  'Upper Tribunal (Tax and Chancery Chamber) List',  '¬© Crown copyright', 'A' , 
        'A v', 'Vladimir Anatolevich Chemukhin for', 'Susan.Woolley@justice.gov.uk', 'Adjourned Bankruptcy Petitions', 'Accessibility', 'Adjudication Enforcement', '-', 'Afterwards', 'Admiralty Application', 'Admiralty CMC', 'Admiralty PTR', 'Admiralty Trial', 
        'Allied Wallet Limited', 'All affected parties shall have permission to apply in respect of any matter concerning the petition.' , 
        'All hearings are adjourned generally with liberty to restore on an urgent basis. If your matter is urgent please e-mail' , 
        'AMTRUST EUROPE LIMITED' , 'and' , 'and MR JUSTICE BUTCHER' ,  'AND UPON' ,  'AND UPON THE COURT being satisfied that the hearing of the petitions cannot presently be conducted remotely and that satisfactory” , “arrangements to ensure safety cannot be put in place such that it is necessary that the Petitions should be adjourned as set',  
        'Appeal' , 'Appeals' , 'Application' ,  'Application & Directions Hearing' ,  'Application and Consequentials/Further Arguments Hearings” , “Application by Andrew Jozef Frejek' ,  'Application by Ashridge Investments Limited' , 'Application by Dell Emerging Markets (EMEA) Ltd and others for', 
        'Application by Folgender Holdings Limited and another for', 'Application by Intergral Petroleum S.A.for', 'Application by Lakatamia Shipping Company Limited for', 
        'Application by Langham Park Homes', 'Application by Navigator Equities Limited and', 'Application by Nicholas Alan Vincent for', 'Application by Sahara Energy Resource Limited for', 'Application by Super-Max Offshore Holdings Limited', 
        'Application by Yuzu Hair Beauty for', 'Application for adjournment', 'Application Hearing & Chancery Trial', 
        'Application Hearing & Costs CMC', 'Application Hearing & Third Party Debt Order', 'as', 'ase contact', 'at', 'At 10:30 AM -', 
        'At 10:30 AM¬†¬†- Remotely via Skype~*', 'Available types:', 'Top ‚Üë', 'Website queries','GOV UK','Help','¬© Crown copyright','Contacts','Privacy','Copyright','GOV.UK','© Crown copyright','Cookies','Young witnesses']

#create new cleaned dataframe 

df_reduce_all = df_reduce_all[df_reduce_all["hearing type"].str.contains('|'.join(bins)) == False].reset_index(drop = True)
df_reduce_all

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,court,Listing,Claimant,Defendant,case_name,judge,date,time,hearing type
0,"(Admiralty, Court, (, QBD, ))","(BL-2019, -, 000207)","(Roman, Production, Limited)","(Silver, Levene, LLP)","[(Roman, Production, Limited), (Silver, Levene...","(DEPUTY, MASTER, LLOYD)","(Wednesday, 23, September, 2020)","(02:30, PM)",Daily court lists
1,"(Business, list, (, ChD, ))","(BL-2018, -, 000586)",(Battenberg),(Phillips),"[(Battenberg), (Phillips)]","(MR, JUSTICE, MILES)","(Wednesday, 23, September, 2020)","(02:00, PM)",Daily court lists
2,"(Commercial, Court, (, QBD, ))","(BR-2015, -, 000780)",(A),"(B, (, skype, ))","[(A), (B, (, skype, ))]","(of, MASTER, CLARK)","(Monday, 8, June, 2020)","(10:30, AM)",*Remote hearing via Skype for Business*
3,"(High, Court, of, Justice)","(BL-2020, -, 000647)",(A),(B),"[(A), (B)]","(MR, JUSTICE, MARCUS, SMITH, –, Via, Skype, *)","(Monday, 8, June, 2020)","(03:00, PM)",Directions Hearing
4,"(Commercial, court)","(PT-2020, -, 000130)",(Ghuman),(Ghuman),"[(Ghuman), (Ghuman)]","(MR, JUSTICE, ANDREW, BAKER)","(Monday, 8, June, 2020)","(10:30, AM)",*Remote hearing via Skype for Business*
...,...,...,...,...,...,...,...,...,...
12300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Legal aid
12301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mercantile Court
12302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pilot forms
12303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Traffic Enforcement Centre


In [32]:
#remove null values 

df_reduce_all_1= df_reduce_all.dropna()
df_reduce_all_1

,court,Listing,Claimant,Defendant,case_name,judge,date,time,hearing type
0,"(Admiralty, Court, (, QBD, ))","(BL-2019, -, 000207)","(Roman, Production, Limited)","(Silver, Levene, LLP)","[(Roman, Production, Limited), (Silver, Levene...","(DEPUTY, MASTER, LLOYD)","(Wednesday, 23, September, 2020)","(02:30, PM)",Daily court lists
1,"(Business, list, (, ChD, ))","(BL-2018, -, 000586)",(Battenberg),(Phillips),"[(Battenberg), (Phillips)]","(MR, JUSTICE, MILES)","(Wednesday, 23, September, 2020)","(02:00, PM)",Daily court lists
2,"(Commercial, Court, (, QBD, ))","(BR-2015, -, 000780)",(A),"(B, (, skype, ))","[(A), (B, (, skype, ))]","(of, MASTER, CLARK)","(Monday, 8, June, 2020)","(10:30, AM)",*Remote hearing via Skype for Business*
3,"(High, Court, of, Justice)","(BL-2020, -, 000647)",(A),(B),"[(A), (B)]","(MR, JUSTICE, MARCUS, SMITH, –, Via, Skype, *)","(Monday, 8, June, 2020)","(03:00, PM)",Directions Hearing
4,"(Commercial, court)","(PT-2020, -, 000130)",(Ghuman),(Ghuman),"[(Ghuman), (Ghuman)]","(MR, JUSTICE, ANDREW, BAKER)","(Monday, 8, June, 2020)","(10:30, AM)",*Remote hearing via Skype for Business*
...,...,...,...,...,...,...,...,...,...
1224,"(Business, and, Property, Courts)","(BL-2020, -, 001098)","(Umbrella, Care, Limited)",(Nisa),"[(Umbrella, Care, Limited), (Nisa)]","(DEPUTY, MASTER, LLOYD, sitting, on, behalf, o...","(Friday, 21, August, 2020)","(11:00, AM)",Costs CMC
1225,"(Business, and, Property, Courts)","(BL-2018, -, 002691)",(A),(B),"[(A), (B)]","(DEPUTY, MASTER, ARKUSH)","(Wednesday, 2, September, 2020)","(11:00, AM)",Commercial court
1226,"(London, Circuit, Commercial, Court)","(HC-2017, -, 001762)","(Turbo, -, K, Limited)","(Turbo, -, K, International, Limited)","[(Turbo, -, K, Limited), (Turbo, -, K, Interna...","(MR, JUSTICE, NUGEE, –, Remotely, via, Skype, *)","(Wednesday, 2, September, 2020)","(11:30, AM)",Remote hearing via Skype for Business
1227,"(Commercial, court, (, QBD, ))","(BL-2019, -, 000353)",(Margossian),(Hunter),"[(Margossian), (Hunter)]","(MR, JUSTICE, PICKEN)","(Wednesday, 2, September, 2020)","(10:30, AM)",Injunction


In [33]:
#remove "()" from dataframe 

df_reduce_all_1["time"] = df_reduce["time"].astype(str).str.replace("(", "")
df_reduce_all_1["date"] = df_reduce["date"].astype(str).str.replace("(", "")
df_reduce_all_1["judge"] = df_reduce["judge"].astype(str).str.replace("(", "")
df_reduce_all_1["case_name"] = df_reduce["case_name"].astype(str).str.replace("(", "")
df_reduce_all_1["Defendant"] = df_reduce["Defendant"].astype(str).str.replace("(", "")
df_reduce_all_1["Claimant"] = df_reduce["Claimant"].astype(str).str.replace("(", "")
df_reduce_all_1["Listing"] = df_reduce["Listing"].astype(str).str.replace("(", "")
df_reduce_all_1["court"] = df_reduce["court"].astype(str).str.replace("(", "")


<ipython-input-33-1b1c6cbf4051>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduce_all_1["time"] = df_reduce["time"].astype(str).str.replace("(", "")
<ipython-input-33-1b1c6cbf4051>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduce_all_1["date"] = df_reduce["date"].astype(str).str.replace("(", "")
<ipython-input-33-1b1c6cbf4051>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [34]:
#save the cleaned dataframe into csv format 
df_reduce_all_1.to_csv('df_reduce_all.csv')


### Step 2: aggregation

In [97]:
#aggregate all the functions together

docs=[]

for filename in glob.glob(os.path.join(folder_path, '*.txt')):
    with open(filename, 'r') as f:
        text = f.read()
        text = text.split('\n')
        for sentence in text:
            if (len(text) !=1):
                newsent=str(sentence)
                doc=nlp(newsent)
                doc=court_name_detector(claimant_defendant_detector(judge_detector(date_detector_1(time_detector_1(hearing_type_generator_fun(doc))))))
                doc.spans
                docs.append(doc)

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname HMRC identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname IN identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname SAR identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  war

### HMM model 


In [98]:
#aggregate the labels of different labelling functions using a generative model

from skweak import aggregation
hmm = skweak.generative.HMM("hmm", ["COURT","LISTING","CLAIMANT ","DEFENDANT ","JUDGE","DATE","TIME","HEARING_TYPE"])
docs = hmm.fit_and_aggregate(docs)

Starting iteration 1
Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Number of processed documents: 13000
Number of processed documents: 14000
Number of processed documents: 15000
Number of processed documents: 16000
Number of processed documents: 17000
Number of processed documents: 18000
Number of processed documents: 19000
Number of processed documents: 20000
Number of processed documents: 21000
Number of processed documents: 22000
Number of processed documents: 23000
Number of processed documents: 24000
Number of processed documents: 25000
Number of processed documents: 26000
Number of processed docume

         1     -345459.4648             +nan


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Number of processed documents: 13000
Number of processed documents: 14000
Number of processed documents: 15000
Number of processed documents: 16000
Number of processed documents: 17000
Number of processed documents: 18000
Number of processed documents: 19000
Number of processed documents: 20000
Number of processed documents: 21000
Number of processed documents: 22000
Number of processed documents: 23000
Number of processed documents: 24000
Number of processed documents: 25000
Number of processed documents: 26000
Number of processed documents: 27000
Number of 

         2     -234668.0469     +110791.4179


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Number of processed documents: 13000
Number of processed documents: 14000
Number of processed documents: 15000
Number of processed documents: 16000
Number of processed documents: 17000
Number of processed documents: 18000
Number of processed documents: 19000
Number of processed documents: 20000
Number of processed documents: 21000
Number of processed documents: 22000
Number of processed documents: 23000
Number of processed documents: 24000
Number of processed documents: 25000
Number of processed documents: 26000
Number of processed documents: 27000
Number of 

         3     -232653.2066       +2014.8403


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Number of processed documents: 13000
Number of processed documents: 14000
Number of processed documents: 15000
Number of processed documents: 16000
Number of processed documents: 17000
Number of processed documents: 18000
Number of processed documents: 19000
Number of processed documents: 20000
Number of processed documents: 21000
Number of processed documents: 22000
Number of processed documents: 23000
Number of processed documents: 24000
Number of processed documents: 25000
Number of processed documents: 26000
Number of processed documents: 27000
Number of 

         4     -232618.1017         +35.1049


In [99]:
#Fit the aggregated model on the training corpus
for doc in docs:
    doc.ents = doc.spans["hmm"]
skweak.utils.docbin_writer(docs, "train_hmm.spacy")
#!python -m spacy train /project/Mishcon_skweak/train_hmm.spacy --output ./output --paths.train train_hmm.spacy --paths.dev train_hmm.spacy

Write to train_hmm.spacy...done


In [100]:
#Fit the aggregated model on the validation corpus

for doc in docs:
    doc.ents = doc.spans["hmm"]
skweak.utils.docbin_writer(docs, "dev_hmm.spacy")

Write to dev_hmm.spacy...done


In [101]:
#Train the final aggregated model on the labelled data.

!spacy init config - --lang en --pipeline ner --optimize accuracy | \
spacy train - --paths.train /project/Mishcon_skweak/train_hmm.spacy  --paths.dev /project/Mishcon_skweak/dev_hmm.spacy \
--initialize.vectors en_core_web_md --output /project/Mishcon_skweak/Models/HMM/


         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.

2022-08-14 23:10:54.394294: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-08-14 23:10:54.394338: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart

 19   13600     104209.60    783.41   98.36   99.47   97.28    0.98
✔ Saved pipeline to output directory
/project/Mishcon_skweak/Models/HMM/model-last


### Voting Model 

In [48]:
#aggregate the labels of different labelling functions using a Majority Voting approach. 

from skweak import aggregation

voter = skweak.voting.SequentialMajorityVoter("maj_voter", labels=["COURT","LISTING","CLAIMANT ","DEFENDANT ","JUDGE","DATE","TIME","HEARING_TYPE"])
docs = voter.fit_and_aggregate(docs)

In [49]:
#Fit the aggregated model on the training corpus

for doc in docs:
    doc.ents = doc.spans["maj_voter"]
skweak.utils.docbin_writer(docs, "train_maj_voter.spacy")

Write to train_maj_voter.spacy...done


In [50]:
#Fit the aggregated model on the validation corpus

for doc in docs:
    doc.ents = doc.spans["maj_voter"]
skweak.utils.docbin_writer(docs, "dev_maj_voter.spacy")

Write to dev_maj_voter.spacy...done


In [51]:
#Train the final aggregated model on the labelled data.

!spacy init config - --lang en --pipeline ner --optimize accuracy | \
spacy train - --paths.train /project/Mishcon_skweak/train_maj_voter.spacy  --paths.dev /project/Mishcon_skweak/dev_maj_voter.spacy \
--initialize.vectors en_core_web_md --output /project/Mishcon_skweak/Models/Voter/


         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.

2022-08-03 12:43:54.673903: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-08-03 12:43:54.673947: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart

In [102]:
#install numpy 
import numpy as np
# install pandas
import pandas
#import utils function from skweak
from skweak import utils


In [103]:
#load the best model 
nlp = spacy.load("/project/Mishcon_skweak/Models/HMM/model-best")


In [104]:
#run it on the document
for filename in glob.glob(os.path.join(folder_path, '*.txt')):
    with open(filename, 'r') as f:
        text = f.read()
        text = text.split('\n')
        #text = text.split('.')
        for sentence in text:
            doc = (nlp(sentence)) 
                # Show an example
            skweak.utils.display_entities(nlp(doc))
    break



In [105]:
#install spacy metrics functions
import spacy
from spacy.training import Example
from spacy.scorer import Scorer

In [106]:
#extract entity with the correct location
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

© Crown copyright 0 17 HEARING_TYPE


In [107]:
#extract entities from all the documents 

abc = []

example = [(ent.text, {'entities':[(ent.start_char, ent.end_char, ent.label_)]})]

for filename in glob.glob(os.path.join(folder_path, '*.txt')):
    with open(filename, 'r') as f:
        text = f.read()
        text = text.split('\n')
        for sentence in text:
            doc = (nlp(sentence)) 
        #text = text.split('.')
            for ent in doc.ents:
                abc.append((ent.text, {'entities':[(ent.start_char, ent.end_char, ent.label_)]}))
                # Show an example
            #skweak.utils.display_entities(nlp(doc))   
        #break

    

In [110]:
#load the best model and evaluate the metrics on each entity 

nlp = spacy.load("/project/Mishcon_skweak/Models/HMM/model-best")
examples = []
scorer = Scorer()
for text, annotations in abc:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annotations)
    example.predicted = nlp(str(example.predicted))
    examples.append(example)
a = scorer.score(examples)
print('ents_p:', a['ents_p'])
print('ents_r:', a['ents_r'])
print('ents_f:', a['ents_f'])
print('ents_per_type:', a['ents_per_type'])

#print(abc)

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "Employment Appeal Tribunal cause list" with entities "[(8, 45, 'COURT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "High Court of Justice" with entities "[(7, 28, 'COURT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "MR JUSTICE MEADE – Remotely via Skype" with entities

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "MR JUSTICE MILES – Remotely via Skype*" with entities "[(7, 45, 'JUDGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "RICHARD SPEARMAN QC sitting as a Judge of the Chan..." with entities "[(7, 92, 'JUDGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "TOM LEECH QC sittin

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "MR JUSTICE WAKSMAN" with entities "[(8, 26, 'JUDGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "PETER MACDONALD EGGERS QC sitting as a Judge of" with entities "[(7, 54, 'JUDGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "HIS HONOUR JUDGE PEARCE sitting as a Judge of

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "MR JUSTICE MILES – Via Skype*" with entities "[(7, 36, 'JUDGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "MASTER KAYE – Remotely via Skype" with entities "[(7, 39, 'JUDGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "DEPUTY MASTER ARKUSH – Remotely via Skype" with e

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "MR JUSTICE MANN – Remotely via Opus 2*" with entities "[(7, 45, 'JUDGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "MR JUSTICE MARCUS SMITH" with entities "[(7, 30, 'JUDGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "HIS HONOUR JUDGE HACON sitting as a Judge of the 

...

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "MRS JUSTICE FALK – Remotely via email *" with entities "[(7, 46, 'JUDGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "DEPUTY MASTER HANSEN remotely via SKYPE for Busine..." with entities "[(7, 59, 'JUDGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "DEPUTY MASTER NURS

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "HHJ PELLING sitting as a Judge of the Technology a..." with entities "[(8, 79, 'JUDGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "SARAH WORTHINGTON QC sitting as a Judge of the Cha..." with entities "[(8, 72, 'JUDGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "11:1

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "HIS HONOUR JUDGE HACON sitting as a Judge of the H..." with entities "[(7, 98, 'JUDGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "MR JUSTICE ADAM JOHNSON" with entities "[(7, 30, 'JUDGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "STUART ISAACS QC sitting as a Judg

/opt/anaconda/envs/Python3/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "MR JUSTICE ZACROLI – Court 10 -Remotely via Skype*" with entities "[(7, 57, 'JUDGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


ents_p: 0.9272177653761191
ents_r: 0.9938048528652556
ents_f: 0.9593572778827978
ents_per_type: {'HEARING_TYPE': {'p': 0.8889950122041813, 'r': 1.0, 'f': 0.9412359550561799}, 'COURT': {'p': 0.97005444646098, 'r': 0.9485359361135759, 'f': 0.9591745177209511}, 'DATE': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'LISTING': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'JUDGE': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'TIME': {'p': 1.0, 'r': 1.0, 'f': 1.0}}
